# Final project: 
# Step 2: Data wrangling

In [711]:
#importar librerias
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import math
import requests
import matplotlib.pyplot as plt
from scipy import stats

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from opencage.geocoder import OpenCageGeocode

### 1. Cities csv scraped from Nomad list (clean and add information to the table)

In [9]:
#importamos csv de ciudades escrapeado de Nomad list
cities = pd.read_csv('../data/nomadlist_cities.csv').drop('Unnamed: 0', axis=1)

#muestra de la tabla
cities.head()

,City,Country,Photo,Overall Score,Quality of life score,Family score,Cost/month (USD),Internet Mbps,Fun,Safety,...,Temperature (°C) Mar,Temperature (°C) Apr,Temperature (°C) May,Temperature (°C) Jun,Temperature (°C) Jul,Temperature (°C) Aug,Temperature (°C) Sep,Temperature (°C) Oct,Temperature (°C) Nov,Temperature (°C) Dec
0,Lisbon,Portugal,https://nomadlist.com/assets/img/places/lisbon...,4.68,4,4,2006.0,27.0,4.0,Great,...,19,18,22,21,23,25,24,21,17,16
1,"Canggu, Bali",Indonesia,https://nomadlist.com/assets/img/places/canggu...,4.61,4,4,1247.0,19.0,4.0,Great,...,30,30,30,28,27,27,27,29,30,31
2,Mexico City,Mexico,https://nomadlist.com/assets/img/places/mexico...,4.46,4,3,1278.0,19.0,4.0,Okay,...,24,24,25,22,21,22,21,21,21,20
3,Chiang Mai,Thailand,https://nomadlist.com/assets/img/places/chiang...,4.45,4,3,1056.0,24.0,4.0,Great,...,36,37,35,32,31,31,31,31,30,28
4,Belgrade,Serbia,https://nomadlist.com/assets/img/places/belgra...,4.45,4,4,1423.0,29.0,4.0,Good,...,15,18,20,28,28,32,26,23,16,8


In [458]:
%%time
#vamos a agregar la latitud, longitud y continente de cada ciudad con el api de OpenCage

#generamos un token para el API (guardado en otro archivo oculto)
from tokens import OpenCage_token

key = OpenCage_token
geocoder = OpenCageGeocode(key)

for i in tqdm(range(300,len(cities))):
    if cities.loc[i,'Continente']!=cities.loc[i,'Continente']:
        try:
            query = cities.loc[i,'City']+', '+cities.loc[i,'Country']
            result = geocoder.geocode(query)
            #print(query)
            for e in result:
                if e['components']['_type']=='city':
                    #print('es ciudad',e['geometry']['lat'],e['geometry']['lng'])
                    cities.loc[i,'lat'] = e['geometry']['lat']
                    cities.loc[i,'lng'] = e['geometry']['lng']
                    cities.loc[i,'Continente'] = e['components']['continent']
                    break
            if cities.loc[i,'Continente']!=cities.loc[i,'Continente']:
                cities.loc[i,'Continente'] = result[0]['components']['continent']
            #print('-----')
            time.sleep(0.3)
        except:
            print(i,query)
    else:
        pass

 19%|█▊        | 189/1020 [00:19<03:17,  4.20it/s]

488 Amundsen-Scott, Antarctica


100%|██████████| 1020/1020 [04:50<00:00,  3.51it/s]

CPU times: user 7.45 s, sys: 593 ms, total: 8.04 s
Wall time: 4min 50s


In [461]:
#dropeamos la ciudad que no fue encontrada
cities.drop(488, inplace=True)
cities.reset_index(drop=True, inplace=True)

In [465]:
#muestra de la tabla incluyendo latitud, longitud y continente
cities.head(3)

,City,Country,Photo,Overall Score,Quality of life score,Family score,Cost/month (USD),Internet Mbps,Fun,Safety,...,Temperature (°C) Aug,Temperature (°C) Sep,Temperature (°C) Oct,Temperature (°C) Nov,Temperature (°C) Dec,lat,lng,Closest Cities (km),Continente,City-Country
0,Lisbon,Portugal,https://nomadlist.com/assets/img/places/lisbon...,4.68,4,4,2006.0,27.0,4.0,Great,...,25,24,21,17,16,38.707751,-9.136592,"{'Ericeira, Portugal': 37, 'Coimbra, Portugal'...",Europe,"Lisbon, Portugal"
1,"Canggu, Bali",Indonesia,https://nomadlist.com/assets/img/places/canggu...,4.61,4,4,1247.0,19.0,4.0,Great,...,27,27,29,30,31,-8.650618,115.136499,"{'Seminyak, Bali, Indonesia': 5, 'Denpasar, Ba...",Asia,"Canggu, Bali, Indonesia"
2,Mexico City,Mexico,https://nomadlist.com/assets/img/places/mexico...,4.46,4,3,1278.0,19.0,4.0,Okay,...,22,21,21,21,20,19.432630,-99.133178,"{'Toluca, Mexico': 57, 'Puebla, Mexico': 107, ...",North America,"Mexico City, Mexico"


In [466]:
#pruebas de lat y lon
key = '94d5c6085cb0421e953a219e122b26bd'
geocoder = OpenCageGeocode(key)
query = 'Montañita, Ecuador'
result = geocoder.geocode(query)

In [469]:
#pruebas de lat y lon
for i in result:
    print(i['components'])
    print(i['confidence'])
    print(i['geometry']['lat'],i['geometry']['lng'])
    print('----')

In [468]:
#pruebas de lat y lon
cities.loc[640,'lat'] = result[2]['geometry']['lat']
cities.loc[640,'lng'] = result[2]['geometry']['lng']

In [303]:
#vamos a agregar una columna que tenga ciudad y pais concatenada

cities['City-Country'] = cities.apply(lambda x: x['City']+', '+x['Country'], axis=1)

In [372]:
#vamos a generar una lista de las ciudades más cercanas a cada una (menos de 1300km)

#función para generar una lista con las ciudades más cercanas a cada una según su latitud y longitud
def closest_cities(row,table):
    rad = 6373.0
    closest_d = []
    closest_c = []
    for i in range(len(table)):
        try:
            lat1 = math.radians(row["lat"])
            lon1 = math.radians(row["lng"])
            lat2 = math.radians(table.loc[i,"lat"])
            lon2 = math.radians(table.loc[i,"lng"])
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
            distance = int(rad * c)
            if distance <= 1300:
                closest_d.append(distance)
                closest_c.append(table.loc[i,'City-Country'])
        except:
            pass  
    zipped = zip(closest_c,closest_d)
    cerc = sorted(zipped, key=lambda tup: tup[1])[1:11]
    #cerc = sorted_zip[1:6]
    return dict(cerc)


In [373]:
#generamos la columna de las ciudades más cercanas a cada ciudad
cities["Closest Cities (km)"] = cities.apply(lambda x: closest_cities(x,cities), axis = 1)

In [374]:
#muestra de la tabla incluyendo ciudades cercanas
cities.head(3)

,City,Country,Photo,Overall Score,Quality of life score,Family score,Cost/month (USD),Internet Mbps,Fun,Safety,...,Temperature (°C) Aug,Temperature (°C) Sep,Temperature (°C) Oct,Temperature (°C) Nov,Temperature (°C) Dec,lat,lng,Closest Cities (km),Continente,City-Country
0,Lisbon,Portugal,https://nomadlist.com/assets/img/places/lisbon...,4.68,4,4,2006.0,27.0,4.0,Great,...,25,24,21,17,16,38.707751,-9.136592,"{'Ericeira, Portugal': 37, 'Coimbra, Portugal'...",Europe,"Lisbon, Portugal"
1,"Canggu, Bali",Indonesia,https://nomadlist.com/assets/img/places/canggu...,4.61,4,4,1247.0,19.0,4.0,Great,...,27,27,29,30,31,-8.650618,115.136499,"{'Seminyak, Bali, Indonesia': 5, 'Denpasar, Ba...",Asia,"Canggu, Bali, Indonesia"
2,Mexico City,Mexico,https://nomadlist.com/assets/img/places/mexico...,4.46,4,3,1278.0,19.0,4.0,Okay,...,22,21,21,21,20,19.432630,-99.133178,"{'Toluca, Mexico': 57, 'Puebla, Mexico': 107, ...",North America,"Mexico City, Mexico"


In [431]:
#No se implemento por problemas técnicos
'''
#vamos a generar una lista de los km en carro que hay entre cada ciudad y sus más cercanas
#se va a usar el api de https://developer.mapquest.com/

#generamos un token para el API (guardado en otro archivo oculto)
from tokens import mapquest_token

#función para generar una lista con las ciudades más cercanas ordenadas según sus km en carro
def closest_by_car(row):
    partida = f"{row['lat']},{row['lng']}"
    #print('start:',partida)
    ciudades = list(row['Closest Cities (km)'].keys())
    indice = list(cities['City-Country']).index(row['City-Country'])
    #print(row['City-Country'],indice)
    closest_dist = []
    closest_time = []
    key = mapquest_token
    
    #iteramos las ciudades cercanas
    for i in ciudades:
        try:
            llegada = f"{cities.loc[cities['City-Country']==i,'lat'].values[0]},{cities.loc[cities['City-Country']==i,'lng'].values[0]}"
            #print('llegada:',llegada)
            url = f'http://open.mapquestapi.com/directions/v2/route?key={key}&from={partida}&to={llegada}&unit=k'
            #peticion al api
            response = requests.get(url)
            #print(response)
            content = response.json()
            distancia = content['route']['distance']
            tiempo = content['route']['formattedTime']
            closest_dist.append((i,distancia))
            closest_time.append((i,tiempo))
        except:
            pass
            print('error: ',row['City-Country'],'---',i)

    closest_dist = dict(sorted(closest_dist, key=lambda tup: tup[1]))
    closest_time = dict(sorted(closest_time, key=lambda tup: tup[1]))
    print(indice, indice/len(cities))
    return {'km':closest_dist,'time':closest_time}
    
cities["Closest Cities by car"] = cities.apply(lambda x: closest_by_car(x), axis = 1)'''


##### Vamos a limpiar los valores nulos y categoricos en la tabla

In [748]:
#vamos a cambiar Safety por valores númericos
safety_dict = {'Great':5,
               'Good':4, 
               'Okay':3,
               'Bad':2,
               'Very bad':1,
               '⚠️ Ongoing armed conflict':1} 

def encode_safety(x):
    try:
        return safety_dict[x]
    except:
        return x

cities['Safety'] = cities['Safety'].apply(lambda x: encode_safety(x))

In [749]:
#columnas con valores nulos antes de limpieza
nans_df = cities.isna().sum().reset_index()
nans_df[nans_df[0]>0]

,index,0
6,Cost/month (USD),130
7,Internet Mbps,6
8,Fun,6
9,Safety,1
10,Education level,43
11,English speaking,3
12,Walkability,1
13,Peace,55
14,Traffic safety,1
15,Hospitals,517


In [753]:
#funcion para rellenar los NaNs con los promedios del mismo pais o continente
def fill_nans(x, columna):
    #validamos si es NaN
    if x[columna]!=x[columna]:
        #llamamos a otras ciudades de otro pais
        mismo_pais = list(cities[(cities['Country']==x['Country'])&(cities[columna].notna())][columna])
        if len(mismo_pais)>0:
            promedio = sum(mismo_pais)/len(mismo_pais)
            return round(promedio)
        else:
            #si no hay elementos de otras ciudades del pais, vamos a rellanar con los promedios del continente
            mismo_continente = list(cities[(cities['Continente']==x['Continente'])&(cities[columna].notna())][columna])
            if len(mismo_continente)>0:
                promedio = sum(mismo_continente)/len(mismo_continente)
                return round(promedio)
            else:
                #si tampoco hay valores de continente, rellenamos con el premedio de toda la columna
                general = list(cities[(cities[columna].notna())][columna])
                promedio = sum(general)/len(general)
                print(columna,x['City'], 'No hay suficientes elementos')
                return round(promedio)
        
    else:
        return x[columna]   
    

In [754]:
#generamos la limpieza de datos
clean_columns = ['Cost/month (USD)', 'Internet Mbps', 'Fun', 'Safety', 'Education level',
                 'English speaking', 'Walkability', 'Peace', 'Traffic safety', 'Hospitals',
                 'Happiness', 'Nightlife', 'Free WiFi in city', 'Places to work from',
                 'Friendly to foreigners', 'Freedom of speech', 'Racial tolerance',
                 'Female friendly', 'LGBTQ friendly', 'Startup Score']

for i in clean_columns:
    cities[i] = cities.apply(lambda x: fill_nans(x, i), axis=1)

Internet Mbps McMurdo Station No hay suficientes elementos
Internet Mbps King Edward Point No hay suficientes elementos
Education level McMurdo Station No hay suficientes elementos
Education level King Edward Point No hay suficientes elementos
Peace McMurdo Station No hay suficientes elementos
Peace King Edward Point No hay suficientes elementos
Hospitals McMurdo Station No hay suficientes elementos
Hospitals King Edward Point No hay suficientes elementos
Happiness McMurdo Station No hay suficientes elementos
Happiness King Edward Point No hay suficientes elementos
Freedom of speech McMurdo Station No hay suficientes elementos
Freedom of speech King Edward Point No hay suficientes elementos


In [768]:
#funcion para rellenar NaNs en columnas de YES/NO con la moda pais o continente
def fill_nansYN(x, columna):
    #validamos si es NaN
    if x[columna]!=x[columna]:
        #llamamos a otras ciudades de otro pais
        mismo_pais = list(cities[(cities['Country']==x['Country'])&(cities[columna].notna())][columna])
        if len(mismo_pais)>0:
            moda = stats.mode(mismo_pais)[0][0]
            #print(x['City'],'pais', moda)
            if moda == 'Yes':
                return 1
            else:
                return 0
        else:
            #si no hay otras ciudades de otro pais, vamos a rellanar con los promedios del continente
            mismo_continente = list(cities[(cities['Continente']==x['Continente'])&(cities[columna].notna())][columna])
            if len(mismo_continente)>0:
                moda = stats.mode(mismo_continente)[0][0]
                #print(x['City'],'continente', moda)
                if moda == 'Yes':
                    return 1
                else:
                    return 0
            else:
                #si tampoco hay valores de continente, rellenamos con la moda de toda la columna
                general = list(cities[(cities[columna].notna())][columna])
                moda = stats.mode(general)[0][0]
                #print(x['City'], columna,'No hay suficientes elementos')
                if moda == 'Yes':
                    return 1
                else:
                    return 0
    else:
        if x[columna] == 'Yes':
            return 1
        else:
            return 0
        return x[columna]   
    

In [769]:
#generamos la limpieza de datos
YN_var = ['Tipping', 'Cashless society', 'Safe tap water']

for i in YN_var:
    cities[i] = cities.apply(lambda x: fill_nansYN(x, i), axis=1)

In [773]:
other_cols = ['Best taxi app in country','Best wireless carrier']

for i in other_cols:
    cities[i] = cities.apply(lambda x: 'No information' if x[i]!=x[i] else x[i], axis=1)

In [774]:
#columnas con valores nulos (ya no hay)
nans_df = cities.isna().sum().reset_index()
nans_df[nans_df[0]>0]

,index,0


In [787]:
#vamos a generar columna con costo promedio por día en USD
cities['Cost/day (USD)'] = round(cities['Cost/month (USD)']/30)

In [788]:
#muestra de la tabla
cities.head(3)

,City,Country,Photo,Overall Score,Quality of life score,Family score,Cost/month (USD),Internet Mbps,Fun,Safety,Education level,English speaking,Walkability,Peace,Traffic safety,Hospitals,Happiness,Nightlife,Free WiFi in city,Places to work from,Friendly to foreigners,Freedom of speech,Racial tolerance,Female friendly,LGBTQ friendly,Startup Score,Best taxi app in country,Best wireless carrier,Tipping,Cashless society,Safe tap water,Population,Temperature (°C) Jan,Temperature (°C) Feb,Temperature (°C) Mar,Temperature (°C) Apr,Temperature (°C) May,Temperature (°C) Jun,Temperature (°C) Jul,Temperature (°C) Aug,Temperature (°C) Sep,Temperature (°C) Oct,Temperature (°C) Nov,Temperature (°C) Dec,lat,lng,Closest Cities (km),Continente,City-Country,Average days,Cost/day (USD)
0,Lisbon,Portugal,https://nomadlist.com/assets/img/places/lisbon...,4.68,4,4,2006.0,27.0,4.0,5.0,3.0,4.0,5.0,4.0,3.0,2.0,3.0,3.0,3.0,5.0,4.0,5.0,3.0,4.0,5.0,3.0,Uber,Vodafone,0,1,1,550000.0,14,16,19,18,22,21,23,25,24,21,17,16,38.707751,-9.136592,"{'Ericeira, Portugal': 37, 'Coimbra, Portugal'...",Europe,"Lisbon, Portugal",4.0,67.0
1,"Canggu, Bali",Indonesia,https://nomadlist.com/assets/img/places/canggu...,4.61,4,4,1247.0,19.0,4.0,5.0,3.0,4.0,3.0,3.0,2.0,3.0,3.0,4.0,4.0,5.0,5.0,3.0,4.0,4.0,4.0,3.0,GoJEK,XL,0,0,0,30000.0,30,30,30,30,30,28,27,27,27,29,30,31,-8.650618,115.136499,"{'Seminyak, Bali, Indonesia': 5, 'Denpasar, Ba...",Asia,"Canggu, Bali, Indonesia",4.0,42.0
2,Mexico City,Mexico,https://nomadlist.com/assets/img/places/mexico...,4.46,4,3,1278.0,19.0,4.0,3.0,3.0,3.0,5.0,2.0,3.0,2.0,4.0,4.0,3.0,5.0,4.0,3.0,2.0,3.0,5.0,3.0,Uber,Telcel,0,0,0,8900000.0,19,22,24,24,25,22,21,22,21,21,21,20,19.432630,-99.133178,"{'Toluca, Mexico': 57, 'Puebla, Mexico': 107, ...",North America,"Mexico City, Mexico",4.0,43.0


In [796]:
#actualizamos el csv con los valores tabla
cities.to_csv('../data/nomadlist_cities.csv',index=False)

### 2. Travelers csv scraped from Nomad list (clean and add information to the table)

In [11]:
#importamos csv de viajeros y las ciudades visitadas escrapeado de Nomad list
users = pd.read_csv('../data/user_cities.csv')

#muestra de la tabla
users.head()

,user,City,Country,Time (days)
0,100marc,"Fort Lauderdale, FL",United States,557
1,100marc,"Miami, FL",United States,475
2,10kjuan,Andorra la Vella,Andorra,1
3,10kjuan,Barcelona,Spain,236
4,10kjuan,Belgrade,Serbia,7


In [112]:
#tamaño de la tabla
users.shape

(35325, 4)

In [114]:
#nulos en la tabla
users.isna().sum()

user           0
City           0
Country        0
Time (days)    0
dtype: int64

In [121]:
#vamos a agregar una columna que tenga ciudad y pais concatenada

users['City-Country'] = users.apply(lambda x: x['City']+', '+x['Country'], axis=1)

In [123]:
#muestra de la tabla
users.head()

,user,City,Country,Time (days),City-Country
0,100marc,"Fort Lauderdale, FL",United States,557,"Fort Lauderdale, FL, United States"
1,100marc,"Miami, FL",United States,475,"Miami, FL, United States"
2,10kjuan,Andorra la Vella,Andorra,1,"Andorra la Vella, Andorra"
3,10kjuan,Barcelona,Spain,236,"Barcelona, Spain"
4,10kjuan,Belgrade,Serbia,7,"Belgrade, Serbia"


In [124]:
#actualizamos el csv con los valores tabla
users.to_csv('../data/user_cities.csv',index=False)

### 3. Cities names in english and spanish

In [244]:
cities_lang = cities[['City','Country']]
cities_lang['City-Country_EN'] = cities_lang.apply(lambda x: x['City']+', '+x['Country'], axis=1)

In [245]:
#tenemos una lista con las ciudades traducidas a español en otro archivo, lo importamos
%reload_ext autoreload
%autoreload 2

from ciudades_en_ES import ciudades_pais_ES
from ciudades_en_ES import ciudades_ES

In [246]:
#agregamos las columnas al dataframe
cities_lang['City_ES'] = ciudades_ES
cities_lang['City-Country_ES'] = ciudades_pais_ES

In [247]:
#agregamos la columna de otros al dataframe con otras opciones de ciudades
cities_lang['Other'] = cities_lang.apply(lambda x: x['City'] if ', ' not in x['City'] else x['City'].split(', ')[0], axis=1)
cities_lang['Other_ES'] = cities_lang.apply(lambda x: x['City_ES'] if ', ' not in x['City_ES'] else x['City'].split(', ')[0], axis=1)

"cities_lang['Other_2'] = cities_lang.apply(lambda x: x['City'] if ', ' not in x['City'] else x['City'].split(', ')[1], axis=1)\ncities_lang['Other_ES_2'] = cities_lang.apply(lambda x: x['City_ES'] if ', ' not in x['City_ES'] else x['City'].split(', ')[1], axis=1)"

In [248]:
#muestra de la tabla
cities_lang

,City,Country,City-Country_EN,City_ES,City-Country_ES,Other,Other_ES
0,Lisbon,Portugal,"Lisbon, Portugal",Lisboa,"Lisboa, Portugal",Lisbon,Lisboa
1,"Canggu, Bali",Indonesia,"Canggu, Bali, Indonesia","Canggu, Bali","Canggu, Bali, Indonesia",Canggu,Canggu
2,Mexico City,Mexico,"Mexico City, Mexico",Ciudad de México,"Ciudad de México, México",Mexico City,Ciudad de México
3,Chiang Mai,Thailand,"Chiang Mai, Thailand",Chiang Mai,"Chiang Mai, Tailandia",Chiang Mai,Chiang Mai
4,Belgrade,Serbia,"Belgrade, Serbia",Belgrado,"Belgrado, Serbia",Belgrade,Belgrado
...,...,...,...,...,...,...,...
1315,"St. Cloud, MN",United States,"St. Cloud, MN, United States","St. Cloud, MN","S t. Cloud, MN, Estados Unidos",St. Cloud,St. Cloud
1316,Volgograd,Russia,"Volgograd, Russia",Volgogrado,"Volgogrado, Rusia",Volgograd,Volgogrado
1317,"Columbia, MO",United States,"Columbia, MO, United States","Columbia, MO","Columbia, MO, Estados Unidos",Columbia,Columbia
1318,Kampala,Uganda,"Kampala, Uganda",Kampala,"Kampala, Uganda",Kampala,Kampala


In [249]:
#generamos el csv con los valores tabla
cities_lang.to_csv('../data/cities_lang.csv',index=False)

### 4. Dataframe that combine all the columns from cities_lang into 1 column (to match cities)

In [251]:
#generamos una tabla que solo traiga una columna con todos los valores y su City_Country_EN
#Esto va a servir para machear ciudades

city_lst = cities_lang[['City','City-Country_EN']]

lst = ['City_ES', 'City-Country_ES',
       'Other', 'Other_ES']

for i in lst:
    city = cities_lang[[i,'City-Country_EN']]
    city.columns = ['City','City-Country_EN']

    city_lst = city_lst.merge(city, how='outer')

In [252]:
#muestra de la tabla
city_lst

,City,City-Country_EN
0,Lisbon,"Lisbon, Portugal"
1,"Canggu, Bali","Canggu, Bali, Indonesia"
2,Mexico City,"Mexico City, Mexico"
3,Chiang Mai,"Chiang Mai, Thailand"
4,Belgrade,"Belgrade, Serbia"
...,...,...
3215,Fairfax,"Fairfax, VA, United States"
3216,Wilmington,"Wilmington, NC, United States"
3217,Flint,"Flint, MI, United States"
3218,St. Cloud,"St. Cloud, MN, United States"


In [253]:
#generamos el csv con los valores tabla
city_lst.to_csv('../data/cities_for_user_input.csv',index=False)

### 5. Average days for each city

In [597]:
#cargamos el dataset
trips = pd.read_csv('../data/total_user_trips.csv')

In [598]:
#vamos a recortar el dataframe para solo tener viajes de máximos de 10 dísa por ciudad
trips = trips[trips['Time (days)']<=10]

In [599]:
#vamos a agregar una columna que tenga ciudad y pais concatenada
trips['City-Country'] = trips.apply(lambda x: x['City']+', '+x['Country'], axis=1)
trips.reset_index(inplace=True)

In [600]:
#muestra de la tabla
trips.head()

,index,user,City,Country,Time (days),City-Country
0,7,goldsaj,Tallinn,Estonia,8,"Tallinn, Estonia"
1,8,goldsaj,Warsaw,Poland,7,"Warsaw, Poland"
2,10,goldsaj,Dubai,United Arab Emirates,9,"Dubai, United Arab Emirates"
3,11,goldsaj,Cairo,Egypt,3,"Cairo, Egypt"
4,12,goldsaj,Alexandria,Egypt,2,"Alexandria, Egypt"


In [601]:
#Generamos matriz de todos las ciudades por días
cities_users = pd.pivot_table(trips,
               index = "index",
               columns = "City-Country",
               values = "Time (days)").fillna(0)

In [607]:
#recortamos los outliers superiores
averages = []
for i in cities_users.columns:
    dataset = list(cities_users[cities_users[i]>0][i].values)
    new_dataset = []
    if len(dataset)>1:
        sorted(dataset)
        q3= np.percentile(dataset,75)
        for e in dataset:
            if e<= q3:
                new_dataset.append(e)
        average = round(sum(new_dataset)/(len(new_dataset)))
        averages.append({'City-Country':i,'Average days':average})
    else:
        averages.append({'City-Country':i,'Average days':3})

In [623]:
#generamos dataframe con los promedios de días por ciudad
avg_days = pd.DataFrame(averages)

In [624]:
#muestra de la tabla
avg_days

,City-Country,Average days
0,"A Coruna, Spain",2.0
1,"Aachen, Germany",2.0
2,"Aalborg, Denmark",2.0
3,"Aarhus, Denmark",2.0
4,"Abbotsford, Canada",1.0
...,...,...
919,"Zandvoort, Netherlands",1.0
920,"Zhangjiakou, China",3.0
921,"Zhuhai, China",3.0
922,"Zurich, Switzerland",2.0


In [625]:
#generamos el csv con los valores tabla
avg_days.to_csv('../data/days_per_city.csv',index=False)

In [646]:
#hacemos el merge con la tabla principal
cities = pd.read_csv('../data/nomadlist_cities.csv')
cities = cities.merge(avg_days, on = 'City-Country', how='left')

In [649]:
#rellenamos las ciudades que no tenian días con 1
cities['Average days'].fillna(1, inplace=True)

In [651]:
#actualizamos el csv con los valores tabla
cities.to_csv('../data/nomadlist_cities.csv',index=False)